In [1]:
# | default_exp schemas.pgsql

In [2]:
%reload_ext autoreload
%autoreload 2
%env AD_MAX_ITEMS=1000
%env AD_SHOW_FULL_STRINGS=1
from torch_snippets import *


env: AD_MAX_ITEMS=1000
env: AD_SHOW_FULL_STRINGS=1


In [3]:
# | export
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.exc import SQLAlchemyError
import datetime
import re
from torch_snippets import track2

In [8]:
# | export
class DbSchemaExplorer:
    def __init__(
        self,
        connection_string: str,
        max_unique_values: int = 30,
        required_tables=None,
        blacklisted_tables=None,
    ):
        if required_tables is None:
            required_tables = []
        self.connection_string = connection_string
        self.max_unique_values = max_unique_values
        self.required_tables = required_tables
        self.blacklisted_tables = blacklisted_tables or []

        self.engine = create_engine(connection_string)
        self.inspector = inspect(self.engine)

        # Discover database name early (best-effort)
        try:
            self.database = self.engine.url.database or ""
            if not self.database:
                with self.engine.connect() as conn:
                    self.database = (
                        conn.execute(text("select current_database()")).scalar() or ""
                    )
        except Exception:
            self.database = ""

        # Default schema (pgsql: public)
        self.default_schema = (
            getattr(self.inspector, "default_schema_name", None) or "public"
        )

        # Tables
        self.table_names = sorted(
            self.inspector.get_table_names(schema=self.default_schema)
        )
        if self.required_tables:
            if isinstance(self.required_tables, str):
                self.table_names = [
                    t for t in self.table_names if re.match(self.required_tables, t)
                ]
            elif isinstance(self.required_tables, list):
                self.table_names = [
                    t for t in self.table_names if t in self.required_tables
                ]
        if self.blacklisted_tables:
            if isinstance(self.blacklisted_tables, str):
                # regex filtering
                self.table_names = [
                    t
                    for t in self.table_names
                    if not re.search(self.blacklisted_tables, t)
                ]
            elif isinstance(self.blacklisted_tables, list):
                self.table_names = [
                    t for t in self.table_names if t not in self.blacklisted_tables
                ]

        # Primary keys map
        self.pk_map = {}
        for t in self.table_names:
            try:
                pk = (
                    self.inspector.get_pk_constraint(t, schema=self.default_schema)
                    or {}
                )
                self.pk_map[t] = set(pk.get("constrained_columns", []) or [])
            except Exception:
                self.pk_map[t] = set()

        # Foreign key references map
        self.fk_ref_map = {t: {} for t in self.table_names}
        for t in self.table_names:
            try:
                fks = (
                    self.inspector.get_foreign_keys(t, schema=self.default_schema) or []
                )
                for fk in fks:
                    cols = fk.get("constrained_columns") or []
                    ref_table = fk.get("referred_table")
                    ref_cols = fk.get("referred_columns") or []
                    for i, c in enumerate(cols):
                        self.fk_ref_map[t][c] = {
                            "table": ref_table,
                            "column": ref_cols[i]
                            if i < len(ref_cols)
                            else (ref_cols[0] if ref_cols else None),
                        }
            except Exception:
                pass

    @staticmethod
    def _iso_utc():
        return datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"

    @staticmethod
    def _quote_ident(s: str) -> str:
        return '"' + s.replace('"', '""') + '"'

    def describe_table(self, table: str) -> dict:
        """Return schema details for a single table."""
        with self.engine.begin() as conn:
            return self._describe_table_with_conn(table, conn)

    def _describe_table_with_conn(self, table: str, conn) -> dict:
        # Row count
        try:
            row_count = int(
                conn.execute(
                    text(
                        f"SELECT COUNT(*) FROM {self._quote_ident(self.default_schema)}.{self._quote_ident(table)}"
                    )
                ).scalar()
                or 0
            )
        except SQLAlchemyError:
            row_count = 0

        # Columns
        columns_info = []
        try:
            cols = self.inspector.get_columns(table, schema=self.default_schema) or []
        except Exception:
            cols = []

        for col in cols:
            col_name = col.get("name")
            data_type = str(col.get("type"))
            nullable = bool(col.get("nullable", True))

            default_val = col.get("default", None)
            if default_val is None:
                default_val = col.get("server_default", None)
            if default_val is not None:
                try:
                    default_val = str(default_val)
                except Exception:
                    default_val = None

            is_pk = col_name in self.pk_map.get(table, set())
            fk_ref = self.fk_ref_map.get(table, {}).get(col_name)
            is_fk = fk_ref is not None

            distinct_count = 0
            unique_values_field = None
            try:
                dq = text(
                    f"SELECT COUNT(DISTINCT {self._quote_ident(col_name)}) "
                    f"FROM {self._quote_ident(self.default_schema)}.{self._quote_ident(table)}"
                )
                dc = conn.execute(dq).scalar()
                distinct_count = int(dc or 0)
                if distinct_count <= self.max_unique_values:
                    uvq = text(
                        f"SELECT DISTINCT {self._quote_ident(col_name)} "
                        f"FROM {self._quote_ident(self.default_schema)}.{self._quote_ident(table)} "
                        f"ORDER BY {self._quote_ident(col_name)} NULLS LAST "
                        f"LIMIT {self.max_unique_values}"
                    )
                    vals = [r[0] for r in conn.execute(uvq).fetchall()]
                    safe_vals = []
                    for v in vals:
                        if isinstance(v, (str, int, float, bool)) or v is None:
                            safe_vals.append(v)
                        else:
                            safe_vals.append(str(v))
                    unique_values_field = safe_vals
            except SQLAlchemyError:
                distinct_count = 0
                unique_values_field = None

            col_entry = {
                "name": col_name,
                "data_type": data_type,
                "nullable": nullable,
                "default": default_val if default_val is not None else None,
                "is_primary_key": is_pk,
                "is_foreign_key": is_fk,
                "references": fk_ref if is_fk else None,
                "distinct_count": distinct_count,
            }
            if unique_values_field is not None:
                col_entry["unique_values"] = unique_values_field

            columns_info.append(col_entry)

        # Indexes
        indexes = []
        try:
            idxs = self.inspector.get_indexes(table, schema=self.default_schema) or []
            for idx in idxs:
                indexes.append({
                    "name": idx.get("name"),
                    "columns": idx.get("column_names") or [],
                    "unique": bool(idx.get("unique", False)),
                })
        except Exception:
            pass

        return {
            "name": table,
            "row_count": row_count,
            "columns": columns_info,
            "indexes": indexes,
        }

    def to_json(self) -> dict:
        result = {
            "database": self.database,
            "generated_at": self._iso_utc(),
            "tables": [],
        }
        with self.engine.begin() as conn:
            for table in track2(self.table_names):
                result["tables"].append(self._describe_table_with_conn(table, conn))
        return result

In [13]:
from dotenv import load_dotenv
import os

load_dotenv("/Users/yeshwanth/Code/Divami/foundry/nl2sql/.env/iirm.env")

user = os.environ["username"]
password = os.environ["password"]
dbhost = os.environ["hostname"]
dbname = os.environ["database"]
connection_string = f"postgresql://{user}:{password}@{dbhost}/{dbname}"

explorer = DbSchemaExplorer(
    connection_string,
    blacklisted_tables="^zz_|^acl_|^notification_|reports|announcement|^audit_",
)
schemas = explorer.to_json()

/var/folders/x_/48jwqq_s1_nbvttnq8z5d3k80000gp/T/ipykernel_53217/578805160.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"


In [10]:
[t for t in explorer.table_names if "reports" in t]

[]

In [11]:
print("default_schema:", explorer.default_schema)
print("tables:", explorer.table_names[:10])

default_schema: public
tables: ['address', 'all_table_user_update_status', 'announcement', 'archived_file_upload', 'broker', 'broker_address', 'broker_contact', 'brokingslip_version_preferred_insurer_detail', 'brokingslip_version_preferred_tpa_detail', 'business_target']


In [12]:
to = "/Users/yeshwanth/Code/Divami/foundry/nl2sql/data/iirm/schema-pgsql.txt"
makedir(parent(to))
AD(schemas).write(to)

In [ ]:
sorted(explorer.table_names)

['address',
 'all_table_user_update_status',
 'announcement',
 'archived_file_upload',
 'broker',
 'broker_address',
 'broker_contact',
 'brokingslip_version_preferred_insurer_detail',
 'brokingslip_version_preferred_tpa_detail',
 'business_target',
 'caution_deposit',
 'caution_deposit_policy_mapping',
 'caution_deposit_transaction',
 'child_details',
 'city',
 'company',
 'company_address',
 'company_contact',
 'company_contact_map',
 'company_detail',
 'company_doc_map',
 'company_strategy',
 'contact',
 'contact_address',
 'contact_communication_details',
 'contact_details',
 'contact_doc_map',
 'country',
 'currency',
 'document',
 'document_processing_file',
 'employee',
 'employee_hierarchy',
 'endorsement',
 'endorsement_field_mapping',
 'file_uploads',
 'final_negotiation_meeting_summary',
 'final_negotiation_participant_map',
 'group_company_map',
 'insurer',
 'insurer_address',
 'insurer_contact',
 'insurer_participant_map',
 'knowledge_central',
 'localization_company_regul

In [ ]:
re.match("reports", "admin_reports_parameters")

In [ ]:
explorer.blacklisted_tables

'^zz_|^acl_|^notification_|reports|^audit_'